In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, GlobalAveragePooling2D
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam

In [ ]:
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
# 경로 설정
data_dir = 'C:\\Users\\intel\\Downloads\\dataset'

# 데이터 증강 및 스케일링
datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.3  # 70% 훈련, 30% 검증+테스트
)

# 데이터 제너레이터 설정
train_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # 70% 훈련 세트
)

validation_generator = datagen.flow_from_directory(
    data_dir,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # 30% 검증+테스트 세트
)

Found 54209 images belonging to 100 classes.
Found 23164 images belonging to 100 classes.


In [5]:
# VGG16 모델 불러오기 (사전 학습된 가중치 사용, 마지막 분류 레이어 제외)
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

In [7]:
# 간단한 모델 생성
model = Sequential([
    base_model,
    GlobalAveragePooling2D(),
    Dense(256, activation='relu'),
    Dense(100, activation='softmax')  # 100개의 클래스로 분류
])

# 모든 층을 학습 가능하게 설정
for layer in base_model.layers:
    layer.trainable = True

In [9]:
# Exponential Decay Example
initial_learning_rate = 0.01
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=initial_learning_rate,
    decay_steps=10000,
    decay_rate=0.9,
    staircase=True)

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [11]:
history = model.fit(
    train_generator,
    epochs=20,
    validation_data=validation_generator
)

Epoch 1/20
1695/1695 [==============================] - 444s 258ms/step - loss: 41299636.0000 - accuracy: 0.0711 - val_loss: 4.1250 - val_accuracy: 0.0756
Epoch 2/20
1695/1695 [==============================] - 430s 254ms/step - loss: 4.1297 - accuracy: 0.0751 - val_loss: 4.1263 - val_accuracy: 0.0756
Epoch 3/20
1695/1695 [==============================] - 428s 252ms/step - loss: 4.1300 - accuracy: 0.0744 - val_loss: 4.1260 - val_accuracy: 0.0756
Epoch 4/20
1695/1695 [==============================] - 427s 252ms/step - loss: 4.1301 - accuracy: 0.0746 - val_loss: 4.1258 - val_accuracy: 0.0756
Epoch 5/20
1695/1695 [==============================] - 431s 254ms/step - loss: 4.1300 - accuracy: 0.0750 - val_loss: 4.1252 - val_accuracy: 0.0756
Epoch 6/20
1695/1695 [==============================] - 449s 265ms/step - loss: 4.1298 - accuracy: 0.0748 - val_loss: 4.1253 - val_accuracy: 0.0756
Epoch 7/20
1695/1695 [==============================] - 499s 294ms/step - loss: 4.1295 - accuracy: 0.0741

KeyboardInterrupt: 

In [1]:
# 시각화
import matplotlib.pyplot as plt

accuracy = history.history["accuracy"]
val_accuracy = history.history["val_accuracy"]
loss = history.history["loss"]
val_loss = history.history["val_loss"]
epochs = range(1, len(accuracy)+1)
plt.plot(epochs, accuracy, "bo", label="Training accuracy")
plt.plot(epochs, val_accuracy, "b", label="Validation accuracy")
plt.title("Training and validation accuracy")
plt.legend()
plt.figure()
plt.plot(epochs, loss, "bo", label="Training loss")
plt.plot(epochs, val_loss, "b", label="Validation loss")
plt.legend()
plt.show()

NameError: name 'history' is not defined